In [11]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd

In [26]:
train = pd.read_csv('data/train.csv', chunksize = 150000) #generator
trained_dfs = []
for i in range(10):
    trained_dfs.append(next(train))

In [27]:
trained_dfs[0].head()

,acoustic_data,time_to_failure
0,12,1.4691
1,6,1.4691
2,8,1.4691
3,5,1.4691
4,8,1.4691


In [28]:
trained_dfs[0].shape

(150000, 2)